In [2]:
import pandas as pd

stores = pd.read_excel('store.xlsx')
devices = pd.read_excel('device.xlsx')
transactions = pd.read_excel('transaction.xlsx')


In [4]:
stores.head()

,id,name,address,city,country,created_at,typology,customer_id
0,1,Vulputate Risus PC,Ap #124-1954 Adipiscing St.,Santa Inês,Belgium,2023-03-24 04:52:21,Florist,22
1,2,Dui Fusce Associates,"Ap #768-3045 Sed, Street",Patarrá,Poland,2023-08-22 03:45:35,Other,30
2,3,Nec Ante Ltd,4587 Tincidunt Avenue,Gaspé,Costa Rica,2022-11-17 10:32:56,FoodTruck,21
3,4,At Auctor Limited,1962 Lorem Avenue,Siquirres,Nigeria,2023-12-15 01:13:46,Hotel,45
4,5,Ac Turpis Limited,"Ap #844-1075 A, Av.",Mazenzele,Ireland,2023-04-13 11:58:34,Restaurant,15


In [5]:
devices.head()

,id,type,store_id
0,1,1,52
1,2,1,84
2,3,3,55
3,4,4,75
4,5,5,58


In [6]:
transactions.head()

,id,device_id,product_name,product_sku,category_name,amount,status,card_number,cvv,created_at,happened_at
0,1,169,libero. Morbi,3770009015048,"amet,",749,cancelled,201433236787273,714,2022-10-16 03:52:35,2022-05-15 08:10:46
1,2,9,"hendrerit a,",3770009015049,magna,235,refused,3477 282862 44521,146,2023-02-22 09:02:54,2022-08-15 09:06:29
2,3,25,neque vitae,3770009015074,"montes,",304,accepted,4556596815473496,562,2023-02-27 06:21:32,2022-01-26 06:50:55
3,4,89,Sed eu,3770009015042,erat,141,accepted,3628 676867 82399,891,2023-01-01 03:10:04,2022-10-09 07:59:34
4,5,40,dolor. Nulla,3770009015049,Etiam,549,cancelled,4903871164699481227,345,2022-10-09 03:20:39,2021-10-23 10:12:44


In [9]:
top_10_stores = transactions[transactions['status'] == 'accepted'].merge(devices, left_on='device_id', right_on='id').merge(stores, left_on='store_id', right_on='id').groupby('name').agg({'amount': 'sum'}).reset_index().sort_values(by='amount', ascending=False).head(10)
print("Top 10 Stores by Transacted Amount:\n", top_10_stores)

Top 10 Stores by Transacted Amount:
                               name  amount
49                    Nec Ante Ltd    9383
64             Sem Ut Cursus Corp.    8909
41                 Magnis Dis Inc.    7334
12            Blandit At Nisi Inc.    7326
78     Volutpat Nunc Sit Institute    6853
42               Mauris Aliquam PC    6739
45      Mauris Sit Amet Associates    6722
73  Ultrices Posuere Cubilia Corp.    6143
59       Pede Ultrices Corporation    5946
61  Sapien Nunc Pulvinar Institute    5926


In [10]:
top_10_products = transactions[transactions['status'] == 'accepted'].groupby('product_name').size().reset_index(name='total_sold').sort_values(by='total_sold', ascending=False).head(10)
print("Top 10 Products Sold:\n", top_10_products)

Top 10 Products Sold:
              product_name  total_sold
94             at, velit.           4
329              sit amet           4
0             Aenean eget           3
367     ultricies ornare,           3
293       posuere cubilia           3
126          dolor. Fusce           3
124             dolor sit           3
113               diam at           3
256     natoque penatibus           3
183  fringilla, porttitor           3


In [11]:
# Average transacted amount per store typology and country
avg_amount_typology_country = transactions[transactions['status'] == 'accepted'].merge(devices, left_on='device_id', right_on='id').merge(stores, left_on='store_id', right_on='id').groupby(['typology', 'country']).agg({'amount': 'mean'}).reset_index()
print("Average Transacted Amount per Store Typology and Country:\n", avg_amount_typology_country)


Average Transacted Amount per Store Typology and Country:
    typology         country      amount
0    Beauty         Austria  584.750000
1    Beauty          Brazil  435.000000
2    Beauty         Ireland  558.000000
3    Beauty           Italy  452.500000
4    Beauty     Netherlands  373.666667
..      ...             ...         ...
62  Service          Mexico  602.000000
63  Service       Singapore  443.333333
64  Service    South Africa  614.666667
65  Service         Ukraine  387.200000
66  Service  United Kingdom  563.375000

[67 rows x 3 columns]


In [12]:
pct_transactions_device_type = transactions[transactions['status'] == 'accepted'].merge(devices, left_on='device_id', right_on='id').groupby('type').size().reset_index(name='count')
total_transactions = pct_transactions_device_type['count'].sum()
pct_transactions_device_type['percentage'] = (pct_transactions_device_type['count'] / total_transactions) * 100
print("Percentage of Transactions per Device Type:\n", pct_transactions_device_type)


Percentage of Transactions per Device Type:
    type  count  percentage
0     1    116   23.770492
1     2     93   19.057377
2     3     88   18.032787
3     4    106   21.721311
4     5     85   17.418033


In [13]:
23.770492 + 19.057377 + 18.032787 + 21.721311 + 17.418033

100.0

In [17]:

# Convertir 'happened_at' a tipo datetime
transactions['happened_at'] = pd.to_datetime(transactions['happened_at'])

# Filtrar transacciones aceptadas y unir con el DataFrame de devices
accepted_transactions = transactions[transactions['status'] == 'accepted'].merge(devices, left_on='device_id', right_on='id')

# Ordenar por 'happened_at' y agrupar por 'store_id'
accepted_transactions = accepted_transactions.sort_values(by='happened_at')
accepted_transactions['rn'] = accepted_transactions.groupby('store_id').cumcount() + 1

# Filtrar tiendas con al menos 5 transacciones
stores_with_5_trans = accepted_transactions[accepted_transactions['rn'] == 5]['store_id'].unique()
filtered_transactions = accepted_transactions[accepted_transactions['store_id'].isin(stores_with_5_trans)]

# Calcular el tiempo en minutos entre la primera y la quinta transacción
first_and_fifth = filtered_transactions[filtered_transactions['rn'].isin([1, 5])]
first_fifth_times = first_and_fifth.pivot(index='store_id', columns='rn', values='happened_at')
first_fifth_times['time_diff_minutes'] = (first_fifth_times[5] - first_fifth_times[1]).dt.total_seconds() / 60

# Calcular el promedio del tiempo en minutos
average_time_minutes = first_fifth_times['time_diff_minutes'].mean()

# Imprimir el resultado
print(f"El tiempo promedio en minutos para que una tienda complete sus primeras 5 transacciones es: {average_time_minutes/(60*24)} dias")


El tiempo promedio en minutos para que una tienda complete sus primeras 5 transacciones es: 243.92414175724636 dias
